# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

from os import listdir, walk
from os.path import isfile, join
from sklearn.preprocessing import MinMaxScaler

# Load Raw Data

In [2]:
%run ../ddc/load_dataset.ipynb

In [3]:
subj_range = np.hstack((np.arange(2001,2002),np.arange(3001,3006)))

all_patients = [str(i) for i in subj_range]

In [4]:
datapath = '../../DDC_Data/'
basepath = '../../'

df_all_p = pd.DataFrame()

for subject_id in all_patients:
    print("Loading {0}'s data".format(subject_id))
    
    acc_filepath = mypath + subject_id + '/' + subject_id + '-log_acc.csv'
    df_raw = pd.read_csv(acc_filepath, header=None, names=['x','y','z','timestamp'])

    df_timer, rec_date, start_time, end_time = load_timer(subject_id)
    df_filt = load_acc(subject_id, rec_date, start_time, end_time)
    df_hr = load_hr(subject_id, rec_date, start_time, end_time)

    df1 = merge_acc_and_hr(df_filt, df_hr)
    
    cols = ['x','y','z']
    xyz_ = df1[cols].to_dict('split')['data']
    xyz_new = MinMaxScaler().fit_transform(xyz_)
    
    for i in range(len(cols)):
        df1[cols[i]] = pd.Series(xyz_new.transpose()[i])

    X_i_p = np.array(df1[cols].to_dict(orient='split')['data'])
    subj_i_p = np.array([subject_id for i in range(X_i_p.shape[0])])
    
    df_all_p = df_all_p.append(df1, sort=False)
    
print('Finished Loading')

Loading 2001's data
Loading 3001's data
Loading 3002's data
Loading 3003's data
Loading 3004's data
Loading 3005's data
Finished Loading


In [5]:
df_all_p = df_all_p.reset_index(drop=True)

# Copy Data

In [45]:
df_day = pd.DataFrame()

In [46]:
T = 0.16
freq = 1/T
oneday = 24*60*60

In [47]:
while(df_day.shape[0]<=int(oneday*freq)):
    df_day = df_day.append(df_all_p, sort=False)
    
df_day.shape

(547074, 6)

In [48]:
df_day = df_day[:int(oneday*freq)]
df_day = df_day.reset_index(drop=True)

df_day['ID'] = pd.Series(['9999' for i in range(df_day.shape[0])])

In [49]:
date = '2019-03-28'
midnight = '00:00:00.000'

time_list = np.array([date + ' ' + calc_ts(calc_sec(midnight)+(T*i)) for i in range(int(oneday*freq))])

In [50]:
time_list.shape

(540000,)

In [51]:
df_day['timestamp'] = pd.Series(time_list)

In [52]:
df_day

ID               timestamp         x         y         z         HR
0       9999      2019-03-28 0:0:0.0  0.246041  0.475436  0.576183  68.205284
1       9999     2019-03-28 0:0:0.16  0.246431  0.473881  0.576429  68.205284
2       9999     2019-03-28 0:0:0.32  0.246236  0.474464  0.573722  68.205284
3       9999     2019-03-28 0:0:0.48  0.246041  0.473881  0.574460  67.365480
4       9999     2019-03-28 0:0:0.64  0.246821  0.472521  0.575444  67.365480
5       9999      2019-03-28 0:0:0.8  0.247016  0.474659  0.572246  67.365480
6       9999     2019-03-28 0:0:0.96  0.247211  0.477574  0.573722  67.365480
7       9999     2019-03-28 0:0:1.12  0.245262  0.469994  0.575198  67.365480
8       9999     2019-03-28 0:0:1.28  0.246041  0.475047  0.571754  67.365480
9       9999     2019-03-28 0:0:1.44  0.246236  0.471937  0.570277  67.365480
10      9999      2019-03-28 0:0:1.6  0.246041  0.473492  0.571508  66.945570
11      9999     2019-03-28 0:0:1.76  0.242144  0.472326  0.572984  66.945570
12      9999     2019-03-28 0:0:1.92  0.245457  0.472132  0.570770  66.945570
13      9999     2019-03-28 0:0:2.08  0.246236  0.475047  0.570277  66.945570
14      9999     2019-03-28 0:0:2.24  0.243703  0.477185  0.573968  66.945570
15      9999      2019-03-28 0:0:2.4  0.244677  0.477963  0.567817  66.945570
16      9999     2019-03-28 0:0:2.56  0.245067  0.475436  0.573476  66.945570
17      9999     2019-03-28 0:0:2.72  0.245262  0.483211  0.566341  66.891680
18      9999     2019-03-28 0:0:2.88  0.247016  0.473492  0.572000  66.891680
19      9999     2019-03-28 0:0:3.04  0.244482  0.471743  0.572738  66.891680
20      9999      2019-03-28 0:0:3.2  0.244482  0.470966  0.573968  66.891680
21      9999     2019-03-28 0:0:3.36  0.244872  0.472715  0.570523  66.891680
22      9999     2019-03-28 0:0:3.52  0.247406  0.473492  0.569785  66.891680
23      9999     2019-03-28 0:0:3.68  0.245652  0.472326  0.574214  66.891680
24      9999     2019-03-28 0:0:3.84  0.246626  0.473881  0.567817  66.883880
25      9999      2019-03-28 0:0:4.0  0.245067  0.474076  0.570770  66.883880
26      9999     2019-03-28 0:0:4.16  0.244677  0.470382  0.572246  66.883880
27      9999     2019-03-28 0:0:4.32  0.243703  0.475825  0.572492  66.883880
28      9999     2019-03-28 0:0:4.48  0.242923  0.470188  0.574460  66.883880
29      9999     2019-03-28 0:0:4.64  0.244872  0.479518  0.571262  66.883880
...      ...                     ...       ...       ...       ...        ...
539970  9999   2019-03-28 23:59:55.2  0.443070  0.601127  0.477337  98.170840
539971  9999  2019-03-28 23:59:55.36  0.443831  0.601687  0.477612  98.170840
539972  9999  2019-03-28 23:59:55.52  0.442499  0.600903  0.477061  98.170840
539973  9999  2019-03-28 23:59:55.68  0.443451  0.601239  0.476097  98.170840
539974  9999  2019-03-28 23:59:55.84  0.441167  0.601127  0.475959  96.996610
539975  9999   2019-03-28 23:59:56.0  0.445354  0.599557  0.474857  96.996610
539976  9999  2019-03-28 23:59:56.16  0.444212  0.591485  0.465627  96.996610
539977  9999  2019-03-28 23:59:56.32  0.418518  0.702141  0.455569  96.996610
539978  9999  2019-03-28 23:59:56.48  0.423086  0.486883  0.153751  96.996610
539979  9999  2019-03-28 23:59:56.64  0.317838  0.589243  0.177309  96.996610
539980  9999   2019-03-28 23:59:56.8  0.365038  0.570183  0.247159  93.627550
539981  9999  2019-03-28 23:59:56.96  0.389209  0.573995  0.250465  93.627550
539982  9999  2019-03-28 23:59:57.12  0.388257  0.575677  0.258456  93.627550
539983  9999  2019-03-28 23:59:57.28  0.388067  0.574444  0.268788  93.627550
539984  9999  2019-03-28 23:59:57.44  0.348670  0.504036  0.239857  93.627550
539985  9999   2019-03-28 23:59:57.6  0.527192  0.654044  0.436006  93.627550
539986  9999  2019-03-28 23:59:57.76  0.450873  0.655950  0.412034  93.627550
539987  9999  2019-03-28 23:59:57.92  0.492363  0.688015  0.525970  93.627550
539988  9999  2019-03-28 23:59:58.08  0.466289  0.644739  0.525281  91.943010
539989  9999  2019-03-28 23:59:5

# Store Cleaned Data in CSV

In [53]:
df_day.to_csv(cleaned_data_path)